# Analysis of exogenous plant status
Routines to evaluate the status of exogenously given power plants and storage units

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Read in data
Read in the following data sets:
* maximum installed capacities for exogenously given power plants
* maximum power output for exogenously given plants per year &rarr; indicates installed power for that particular year
* capacities for exogenously given storage units

In [ ]:
path_folder = "./model_inputs/"
file_name_transformers = "transformers_exogenous.csv"
file_name_transformers_max = "transformers_exogenous_max_ts.csv"
file_name_storages = "storages_el_exogenous.csv"

FUELS = {
    "biomass": "#15b01a",
    "uranium": "#e50000",
    "lignite": "#7f2b0a",
    "hardcoal": "#000000",
    "natgas": "#ffd966",
    "hydrogen": "#6fa8dc",
    "mixedfuels": "#a57e52",
    "otherfossil": "#d8dcd6",
    "waste": "#c04e01",
    "oil": "#aaa662",
}

STORAGES = {
    "DE_storage_el_PHS": "#0c2aac",
}

## Rearrange and plot data for transformers
* Combine time series and maximum installed capacities to obtain capacities per year
* Groupby fuel and year
* Draw and show plot

In [ ]:
transformers_capacities = pd.read_csv(f"{path_folder}{file_name_transformers}", index_col=0)["capacity"]
transformers_max_ts = pd.read_csv(f"{path_folder}{file_name_transformers_max}", index_col=0)

# Combine information to derive installed capacities by year
transformers_capacity_ts = transformers_max_ts.mul(transformers_capacities)

# Do some reformatting
transformers_capacity_ts.index = transformers_capacity_ts.index.str[:4]
transformers_capacity_ts_transposed = transformers_capacity_ts.T
transformers_capacity_ts_transposed.index = transformers_capacity_ts_transposed.index.str.split("_", expand=True)
transformers_capacity_ts_transposed.index.names = ["component_name", "fuel", "type", "cluster", "cluster_no"]
transformers_capacity_ts_transposed.reset_index(inplace=True)

# groupby fuel
capacity_by_fuel_year =  transformers_capacity_ts_transposed.groupby("fuel").sum().T

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
capacity_by_fuel_year = capacity_by_fuel_year[[col for col in FUELS.keys()]]
_ = capacity_by_fuel_year.plot(kind="bar", stacked=True, ax=ax, color=FUELS)
_ = plt.xlabel("year")
_ = plt.ylabel("installed capacity")
_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
plt.show()

## Rearrange and plot data for storages
* Read in data
* Reshape and repeat since installed values are kept constant

In [ ]:
storages = pd.read_csv(f"{path_folder}{file_name_storages}", index_col=0)
storages = storages.loc[
    storages["country"] == "DE", 
    ["capacity_pump", "capacity_turbine", "nominal_storable_energy"]
]
storages.rename(
    columns={"capacity_pump": "inflow_power", "capacity_turbine": "outflow_power"},
    inplace=True
)

In [ ]:
storages = pd.DataFrame(
    index=range(2020, 2051), 
    columns=storages["inflow_power"].index, 
    data={
        storages["inflow_power"].index[0]: storages["inflow_power"].values[0],
    }
)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
storages = storages[[col for col in STORAGES.keys()]]
_ = storages.plot(kind="bar", stacked=True, ax=ax, color=STORAGES)
_ = plt.xlabel("year")
_ = plt.ylabel("installed capacity")
_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
plt.show()